<a href="https://colab.research.google.com/github/dookda/cmu_lab_154743/blob/main/python_workshop/python_15_raster_geospatial_on_cloud_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap
# conda install -c conda-forge earthengine-api
# conda install geemap

In [ ]:
import ee
import geemap

# Authenticate the Earth Engine library.
try:
    ee.Initialize(project="ee-sakda-451407")  # uses cached credentials if available
except Exception:
    ee.Authenticate()
    ee.Initialize(project="ee-sakda-451407")


In [ ]:
import ee
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# ตั้งค่า matplotlib สำหรับการแสดงผลภาษาไทย (ถ้าต้องการ)
plt.rcParams['font.family'] = 'DejaVu Sans'

class CHIRPSRainfallPredictor:
    def __init__(self, aoi):
        """
        เริ่มต้นการทำนายปริมาณฝนด้วยพื้นที่สนใจ

        Args:
            aoi: ee.Geometry object ที่กำหนดพื้นที่สนใจ
        """
        self.aoi = aoi
        self.scaler = MinMaxScaler()  # สำหรับการปรับมาตรฐานข้อมูล
        self.model = None  # โมเดล LSTM
        self.data = None   # ข้อมูลฝนดิบ
        self.scaled_data = None  # ข้อมูลฝนที่ปรับมาตรฐานแล้ว

    def extract_chirps_data(self, start_date='1981-01-01', end_date=None):
        """
        ดึงข้อมูลปริมาณฝน CHIRPS สำหรับพื้นที่และช่วงวันที่ที่กำหนด

        Args:
            start_date (str): วันที่เริ่มต้น รูปแบบ 'YYYY-MM-DD'
            end_date (str): วันที่สิ้นสุด รูปแบบ 'YYYY-MM-DD' (ค่าเริ่มต้น: เมื่อวาน)
        """
        if end_date is None:
            end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

        print(f"กำลังดึงข้อมูล CHIRPS ตั้งแต่ {start_date} ถึง {end_date}")

        # โหลดข้อมูล CHIRPS dataset
        chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
                   .filter(ee.Filter.date(start_date, end_date)) \
                   .select('precipitation')

        # ฟังก์ชันสำหรับดึงค่าเฉลี่ยปริมาณฝนในแต่ละวัน
        def extract_rainfall(image):
            # คำนวณค่าเฉลี่ยปริมาณฝนในพื้นที่ AOI
            mean_precip = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=self.aoi,
                scale=5566,  # ความละเอียดดั้งเดิมของ CHIRPS ~5.5km
                maxPixels=1e9
            )

            # ดึงวันที่
            date = image.date().format('YYYY-MM-dd')

            return ee.Feature(None, {
                'date': date,
                'precipitation': mean_precip.get('precipitation')
            })

        # ประยุกต์ใช้ฟังก์ชันกับทุกภาพในชุดข้อมูล
        rainfall_data = chirps.map(extract_rainfall)

        # แปลงเป็น pandas DataFrame
        data_list = rainfall_data.getInfo()['features']

        # ประมวลผลข้อมูล
        dates = []
        precipitation = []

        for feature in data_list:
            props = feature['properties']
            if props['precipitation'] is not None:
                dates.append(pd.to_datetime(props['date']))
                precipitation.append(float(props['precipitation']))

        # สร้าง DataFrame
        self.data = pd.DataFrame({
            'date': dates,
            'precipitation': precipitation
        }).sort_values('date').reset_index(drop=True)

        print(f"ดึงข้อมูลปริมาณฝนสำเร็จ {len(self.data)} วัน")
        return self.data

    def preprocess_data(self, sequence_length=30):
        """
        ประมวลผลข้อมูลล่วงหน้าสำหรับการฝึกโมเดล LSTM

        Args:
            sequence_length (int): จำนวนวันที่ใช้สำหรับการทำนาย
        """
        if self.data is None:
            raise ValueError("ไม่มีข้อมูล กรุณารัน extract_chirps_data() ก่อน")

        # จัดการค่าที่หายไป (ใส่ 0 แทน)
        self.data['precipitation'].fillna(0, inplace=True)

        # ปรับมาตรฐานข้อมูล (Normalize)
        precipitation_values = self.data['precipitation'].values.reshape(-1, 1)
        self.scaled_data = self.scaler.fit_transform(precipitation_values)

        # สร้างลำดับข้อมูลสำหรับ LSTM
        X, y = [], []
        for i in range(sequence_length, len(self.scaled_data)):
            X.append(self.scaled_data[i-sequence_length:i, 0])  # ข้อมูล input
            y.append(self.scaled_data[i, 0])  # ข้อมูล target

        self.X = np.array(X)
        self.y = np.array(y)

        # ปรับรูปแบบ X สำหรับ LSTM input (samples, timesteps, features)
        self.X = self.X.reshape((self.X.shape[0], self.X.shape[1], 1))

        print(f"ประมวลผลข้อมูลเสร็จสิ้น: {self.X.shape[0]} ตัวอย่างด้วยลำดับ {sequence_length} วัน")

    def split_data(self, train_ratio=0.8):
        """แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ"""
        split_idx = int(len(self.X) * train_ratio)

        self.X_train = self.X[:split_idx]
        self.y_train = self.y[:split_idx]
        self.X_test = self.X[split_idx:]
        self.y_test = self.y[split_idx:]

        print(f"แบ่งข้อมูล: {len(self.X_train)} ตัวอย่างสำหรับฝึก, {len(self.X_test)} ตัวอย่างสำหรับทดสอบ")

    def build_lstm_model(self, sequence_length=30, lstm_units=[50, 50], dropout_rate=0.2):
        """
        สร้างโมเดล LSTM สำหรับการทำนายปริมาณฝน

        Args:
            sequence_length (int): ความยาวของลำดับ input
            lstm_units (list): จำนวน units ในแต่ละชั้น LSTM
            dropout_rate (float): อัตรา dropout สำหรับป้องกัน overfitting
        """
        self.model = Sequential()

        # ชั้น LSTM แรก
        self.model.add(LSTM(lstm_units[0],
                           return_sequences=True if len(lstm_units) > 1 else False,
                           input_shape=(sequence_length, 1)))
        self.model.add(Dropout(dropout_rate))

        # ชั้น LSTM เพิ่มเติม
        for i in range(1, len(lstm_units)):
            return_seq = True if i < len(lstm_units) - 1 else False
            self.model.add(LSTM(lstm_units[i], return_sequences=return_seq))
            self.model.add(Dropout(dropout_rate))

        # ชั้น output
        self.model.add(Dense(1))

        # คอมไพล์โมเดล
        self.model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        print("สร้างโมเดล LSTM เรียบร้อยแล้ว")
        print(self.model.summary())

    def train_model(self, epochs=100, batch_size=32, validation_split=0.2):
        """ฝึกโมเดล LSTM"""
        if self.model is None:
            raise ValueError("ยังไม่ได้สร้างโมเดล กรุณารัน build_lstm_model() ก่อน")

        # Early stopping callback เพื่อหยุดการฝึกเมื่อไม่มีการปรับปรุง
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        # ฝึกโมเดล
        history = self.model.fit(
            self.X_train, self.y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=[early_stopping],
            verbose=1
        )

        return history

    def evaluate_model(self):
        """ประเมินประสิทธิภาพของโมเดลบนชุดทดสอบ"""
        if self.model is None:
            raise ValueError("ยังไม่ได้ฝึกโมเดล")

        # ทำการทำนาย
        y_pred = self.model.predict(self.X_test)

        # แปลงค่าทำนายกลับเป็นค่าจริง (inverse transform)
        y_pred_actual = self.scaler.inverse_transform(y_pred)
        y_test_actual = self.scaler.inverse_transform(self.y_test.reshape(-1, 1))

        # คำนวณค่าตัวชี้วัดประสิทธิภาพ
        mse = mean_squared_error(y_test_actual, y_pred_actual)
        mae = mean_absolute_error(y_test_actual, y_pred_actual)
        r2 = r2_score(y_test_actual, y_pred_actual)  # เพิ่ม R² Score
        rmse = np.sqrt(mse)

        print(f"ประสิทธิภาพของโมเดล:")
        print(f"MSE (Mean Squared Error): {mse:.4f}")
        print(f"MAE (Mean Absolute Error): {mae:.4f}")
        print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
        print(f"R² Score (Coefficient of Determination): {r2:.4f}")

        return {
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,  # เพิ่ม R² ในผลลัพธ์
            'predictions': y_pred_actual,
            'actual': y_test_actual
        }

    def predict_next_month(self, days=30):
        """
        ทำนายปริมาณฝนสำหรับเดือนถัดไป

        Args:
            days (int): จำนวนวันที่ต้องการทำนาย
        """
        if self.model is None:
            raise ValueError("ยังไม่ได้ฝึกโมเดล")

        # ใช้ลำดับสุดท้ายจากข้อมูลสำหรับการทำนาย
        last_sequence = self.scaled_data[-30:].reshape(1, 30, 1)

        predictions = []
        current_sequence = last_sequence.copy()

        for _ in range(days):
            # ทำนายวันถัดไป
            next_pred = self.model.predict(current_sequence, verbose=0)
            predictions.append(next_pred[0, 0])

            # อัพเดทลำดับสำหรับการทำนายครั้งถัดไป (sliding window)
            current_sequence = np.roll(current_sequence, -1, axis=1)
            current_sequence[0, -1, 0] = next_pred[0, 0]

        # แปลงค่าทำนายกลับเป็นค่าจริง (inverse transform)
        predictions_array = np.array(predictions).reshape(-1, 1)
        predicted_rainfall = self.scaler.inverse_transform(predictions_array)

        # สร้างวันที่สำหรับอนาคต
        last_date = self.data['date'].iloc[-1]
        future_dates = pd.date_range(
            start=last_date + timedelta(days=1),
            periods=days,
            freq='D'
        )

        # สร้าง DataFrame ของผลการทำนาย
        prediction_df = pd.DataFrame({
            'date': future_dates,
            'predicted_precipitation': predicted_rainfall.flatten()
        })

        return prediction_df

    def plot_results(self, evaluation_results=None, predictions=None, show_recent_days=365):
        """แสดงผลกราฟข้อมูลประวัติ ประสิทธิภาพโมเดล และการทำนาย"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))

        # กราฟที่ 1: ข้อมูลปริมาณฝนล่าสุด
        recent_data = self.data.tail(show_recent_days)
        axes[0, 0].plot(recent_data['date'], recent_data['precipitation'], color='blue', alpha=0.7)
        axes[0, 0].set_title('Historical Rainfall Data', fontsize=14)
        axes[0, 0].set_xlabel('Date')
        axes[0, 0].set_ylabel('Precipitation (mm)')
        axes[0, 0].tick_params(axis='x', rotation=45)
        axes[0, 0].grid(True, alpha=0.3)

        # กราฟที่ 2: ประสิทธิภาพโมเดล (ถ้ามี)
        if evaluation_results:
            axes[0, 1].scatter(evaluation_results['actual'],
                              evaluation_results['predictions'], alpha=0.6, color='blue')
            # เส้นแสดงความสมบูรณ์แบบ (perfect prediction line)
            max_val = max(evaluation_results['actual'].max(), evaluation_results['predictions'].max())
            axes[0, 1].plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Perfect Prediction')
            axes[0, 1].set_title(f'Actual vs Predicted (R² = {evaluation_results["r2"]:.3f})', fontsize=14)
            axes[0, 1].set_xlabel('Actual Precipitation (mm)')
            axes[0, 1].set_ylabel('Predicted Precipitation (mm)')
            axes[0, 1].grid(True, alpha=0.3)
            axes[0, 1].legend()
        else:
            axes[0, 1].text(0.5, 0.5, 'No Model Evaluation Available',
                           horizontalalignment='center', verticalalignment='center',
                           transform=axes[0, 1].transAxes, fontsize=12)
            axes[0, 1].set_title('Model Performance')

        # กราฟที่ 3: ผลการทำนาย
        if predictions is not None:
            # แสดงข้อมูลย้อนหลัง 60 วัน + การทำนาย
            recent_hist = self.data.tail(60)
            axes[1, 0].plot(recent_hist['date'], recent_hist['precipitation'],
                           label='Historical', color='blue', linewidth=2)
            axes[1, 0].plot(predictions['date'], predictions['predicted_precipitation'],
                           label='Predicted', color='red', linestyle='--', linewidth=2)
            axes[1, 0].set_title('Historical vs Predicted Rainfall', fontsize=14)
            axes[1, 0].set_xlabel('Date')
            axes[1, 0].set_ylabel('Precipitation (mm)')
            axes[1, 0].legend()
            axes[1, 0].tick_params(axis='x', rotation=45)
            axes[1, 0].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'No Predictions Available',
                           horizontalalignment='center', verticalalignment='center',
                           transform=axes[1, 0].transAxes, fontsize=12)
            axes[1, 0].set_title('Rainfall Predictions')

        # กราฟที่ 4: ค่าเฉลี่ยรายเดือน
        monthly_avg = self.data.copy()
        monthly_avg['month'] = monthly_avg['date'].dt.month
        monthly_precip = monthly_avg.groupby('month')['precipitation'].mean()

        months_eng = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

        bars = axes[1, 1].bar(range(1, 13), monthly_precip.values, color='skyblue', alpha=0.7)
        axes[1, 1].set_title('Average Monthly Rainfall', fontsize=14)
        axes[1, 1].set_xlabel('Month')
        axes[1, 1].set_ylabel('Average Precipitation (mm)')
        axes[1, 1].set_xticks(range(1, 13))
        axes[1, 1].set_xticklabels(months_eng)
        axes[1, 1].grid(True, alpha=0.3, axis='y')

        # เพิ่มค่าบนแต่ละแท่ง
        for i, bar in enumerate(bars):
            height = bar.get_height()
            axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.1,
                           f'{height:.1f}', ha='center', va='bottom', fontsize=10)

        plt.tight_layout()
        plt.show()

# ตัวอย่างการใช้งาน
def main():
    """ฟังก์ชันหลักสำหรับรันระบบทำนายปริมาณฝน"""
    # กำหนดพื้นที่สนใจ (พิกัดของคุณ)
    aoi = ee.Geometry.Polygon([[[98.8, 18.6], [99.2, 18.6], [99.2, 19.1], [98.8, 19.1], [98.8, 18.6]]])

    # เริ่มต้นการทำนาย
    predictor = CHIRPSRainfallPredictor(aoi)

    try:
        # ขั้นที่ 1: ดึงข้อมูล CHIRPS
        print("ขั้นที่ 1: กำลังดึงข้อมูล CHIRPS...")
        data = predictor.extract_chirps_data(start_date='2022-01-01')  # ปรับช่วงวันที่ตามต้องการ

        if len(data) < 100:  # ตรวจสอบว่ามีข้อมูลเพียงพอหรือไม่
            print("คำเตือน: ข้อมูลมีน้อยเกินไป อาจส่งผลต่อประสิทธิภาพของโมเดล")

        # ขั้นที่ 2: ประมวลผลข้อมูลล่วงหน้า
        print("\nขั้นที่ 2: กำลังประมวลผลข้อมูล...")
        predictor.preprocess_data(sequence_length=30)
        predictor.split_data(train_ratio=0.8)

        # ขั้นที่ 3: สร้างโมเดล LSTM
        print("\nขั้นที่ 3: กำลังสร้างโมเดล LSTM...")
        predictor.build_lstm_model(sequence_length=30, lstm_units=[64, 32])

        # ขั้นที่ 4: ฝึกโมเดล
        print("\nขั้นที่ 4: กำลังฝึกโมเดล...")
        history = predictor.train_model(epochs=50, batch_size=32)

        # ขั้นที่ 5: ประเมินโมเดล
        print("\nขั้นที่ 5: กำลังประเมินโมเดล...")
        eval_results = predictor.evaluate_model()

        # ขั้นที่ 6: ทำการทำนาย
        print("\nขั้นที่ 6: กำลังทำนาย 30 วันข้างหน้า...")
        future_predictions = predictor.predict_next_month(days=30)

        # แสดงผลลัพธ์
        print("\n" + "="*50)
        print("ผลการทำนายปริมาณฝน 30 วันข้างหน้า:")
        print("="*50)
        print(future_predictions.head(10))
        print("...")
        print(f"ปริมาณฝนรวมที่คาดการณ์สำหรับเดือนหน้า: {future_predictions['predicted_precipitation'].sum():.2f} มม.")
        print(f"ปริมาณฝนเฉลี่ยต่อวัน: {future_predictions['predicted_precipitation'].mean():.2f} มม.")
        print(f"ปริมาณฝนสูงสุด: {future_predictions['predicted_precipitation'].max():.2f} มม.")
        print(f"ปริมาณฝนต่ำสุด: {future_predictions['predicted_precipitation'].min():.2f} มม.")

        # สร้างกราฟแสดงผล
        print("\nกำลังสร้างกราฟแสดงผล...")
        predictor.plot_results(evaluation_results=eval_results, predictions=future_predictions)

        return predictor, future_predictions

    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {str(e)}")
        return None, None

# ฟังก์ชันสำหรับบันทึกผลลัพธ์
def save_results(predictions, filename="rainfall_predictions.csv"):
    """
    บันทึกผลการทำนายลงไฟล์ CSV

    Args:
        predictions: DataFrame ของผลการทำนาย
        filename: ชื่อไฟล์ที่ต้องการบันทึก
    """
    if predictions is not None:
        predictions.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"บันทึกผลการทำนายลงไฟล์ {filename} เรียบร้อยแล้ว")
    else:
        print("ไม่มีผลการทำนายให้บันทึก")

if __name__ == "__main__":
    # รันฟังก์ชันหลัก
    print("เริ่มต้นระบบทำนายปริมาณฝนจาก CHIRPS ด้วย LSTM")
    print("="*60)

    predictor, predictions = main()

    if predictions is not None:
        # ถามผู้ใช้ว่าต้องการบันทึกผลลัพธ์หรือไม่
        save_choice = input("\nต้องการบันทึกผลการทำนายลงไฟล์ CSV หรือไม่? (y/n): ")
        if save_choice.lower() == 'y':
            filename = input("ระบุชื่อไฟล์ (กด Enter สำหรับชื่อเริ่มต้น): ")
            if filename.strip() == "":
                filename = "rainfall_predictions.csv"
            save_results(predictions, filename)

    print("\nการทำงานเสร็จสิ้น!")